<a href="https://colab.research.google.com/github/sellinkaya/T-bbi-Sigorta-demesi-ile-Makine-renmesi-Projesi/blob/main/Tibbi_Sigorta_%C3%96demesi_Projesi_adl%C4%B1_not_defteri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

# Gerekli kütüphanelerin yüklenmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score



Saving insurance.csv to insurance (1).csv


İnceleyeceğiniz Python kodu, bir sigorta veri kümesi üzerinde keşifsel veri analizi (EDA) yapmak, verileri ön işlemek ve makine öğrenimi modelleri için hazırlamak amacıyla yazılmıştır.

Veri Seti Hakkında
ACME Insurance Inc., Amerika Birleşik Devletleri'nin her yerindeki binlerce müşteriye uygun fiyatlı sağlık sigortası sunmaktadır. Yaş, cinsiyet, BMI, çocuklar, sigara alışkanlıkları ve ikamet ettikleri bölge gibi bilgileri kullanarak yeni müşteriler için yıllık tıbbi harcamaları tahmin eden otomatik bir sistem oluşturmak hedeflenmiştir.


In [2]:
import warnings
warnings.filterwarnings('ignore')

import warnings ve warnings.filterwarnings('ignore') satırları, Python kodunuz çalışırken ortaya çıkabilecek uyarı mesajlarını (warnings) gizlemek amacıyla kullanılır.

In [3]:
# Veri Yükleme
df = pd.read_csv('insurance.csv', sep=';')
df.info()

#Eksik veri kontrolü yapıldı
df.isnull().any().any()
df.head(10)
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 73.3+ KB


,age,bmi,children
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918
std,14.049960,6.098187,1.205493
min,18.000000,15.960000,0.000000
25%,27.000000,26.296250,0.000000
50%,39.000000,30.400000,1.000000
75%,51.000000,34.693750,2.000000
max,64.000000,53.130000,5.000000


# New Section

 Kütüphane Yükleme ve Veri İnceleme
Kütüphaneler: numpy (sayısal işlemler), pandas (veri manipülasyonu), matplotlib.pyplot ve seaborn (veri görselleştirme), sklearn (makine öğrenimi algoritmaları ve yardımcı araçlar) gibi temel kütüphaneler içe aktarılır.

Veri Yükleme: insurance.csv adlı CSV dosyası pd.read_csv('insurance.csv', sep=';') komutuyla yüklenir. sep=';' ifadesi, dosyanın noktalı virgülle ayrılmış olduğunu belirtir.

İlk İnceleme:

df.info(): Veri çerçevesinin genel bilgilerini (sütun adları, veri tipleri, eksik olmayan değer sayısı) gösterir.

df.isnull().sum(): Her sütundaki eksik değer sayısını kontrol eder.

df.head(): Veri çerçevesinin ilk 5 satırını görüntüler.

df.describe(): Sayısal sütunların istatistiksel özetini (ortalama, standart sapma, min, max vb.) sunar.

In [4]:

# 'charges' sütununu sayısala çevirme
# (Nokta yerine boşluk koyup virgül gibi davranmasını engelleme)
df['charges'] = df['charges'].str.replace('.', '', regex=False).astype(float)

# One-hot encoding uygulama
df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)

# Özellikler ve hedef değişken
X = df_encoded.drop('charges', axis=1)
y = df_encoded['charges']

# Sadece sayısal sütunları seç
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns


Veri Temizleme ve Ön İşleme
'charges' Sütunu Dönüşümü: df['charges'] = df['charges'].str.replace('.', '', regex=False).astype(float) satırı, 'charges' (sigorta masrafları) sütunundaki ondalık ayırıcı olarak kullanılan noktaları kaldırır ve sütunu float (ondalıklı sayı) veri tipine dönüştürür. Bu, büyük sayılarda binlik ayıracı olarak nokta kullanıldığı durumlarda veri tipini doğru ayarlamak için önemlidir.

One-Hot Encoding: df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True) ile kategorik sütunlar ('sex', 'smoker', 'region') one-hot encoding'e tabi tutulur. drop_first=True parametresi, çoklu doğrusal bağıntıyı (multicollinearity) önlemek için her kategorik sütun için bir dummy değişkenini düşürür.

Özellikler ve Hedef Değişken: X (bağımsız değişkenler) ve y (bağımlı değişken - 'charges') ayrılır.

In [5]:
# Aykırı Değer Filtreleme
q01 = df['charges'].quantile(0.01)
q99 = df['charges'].quantile(0.99)
df_filtered = df[(df['charges'] > q01) & (df['charges'] < q99)]


Aykırı Değer Filtreleme
charges sütunu için %1 ve %99'luk çeyreklikleri (quantile) kullanarak aykırı değerler filtrelenir. Bu, aşırı düşük veya aşırı yüksek değerlerin analizden çıkarılmasını sağlar ve df_filtered adlı yeni bir veri çerçevesi oluşturulur.

In [ ]:
# Boxplot çizimi
plt.figure(figsize=(11, 5))
for i, column in enumerate(numeric_columns):
    plt.subplot(1, len(numeric_columns), i + 1)
    sns.boxplot(y=df[column])
    plt.title(column)
plt.tight_layout()
plt.show()

In [ ]:
# Dağılım grafiği
plt.figure(figsize=(10, 5))
sns.histplot(df['charges'], kde=True, color='skyblue')
plt.title('Quality Distribution Plot', fontsize=15)
plt.show()


In [ ]:
# Değişkenler Arası İlişki Matrisi (Pair Plot)
sns_plot = sns.pairplot(df)

In [ ]:

# Saçılım Grafikleri Oluşturma

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.scatterplot(x='age', y='charges', hue='smoker', data=df, palette='viridis')
plt.title('Charges vs. Age')

plt.subplot(1, 3, 2)
sns.scatterplot(x='bmi', y='charges', hue='smoker', data=df, palette='viridis')
plt.title('Charges vs. BMI')

plt.subplot(1, 3, 3)
sns.scatterplot(x='children', y='charges', hue='smoker', data=df, palette='viridis')
plt.title('Charges vs. Children')
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Pairplot
plt.figure(figsize=(10, 8))
sns.pairplot(df, hue='smoker', vars=['age', 'bmi', 'children', 'charges'])
plt.show()

# Heatmap için sadece sayısal sütunları al
numeric_df = df.select_dtypes(include=['number'])

plt.figure(figsize=(8, 6))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()

Keşifsel Veri Analizi (EDA) - Görselleştirmeler
Boxplotlar: Sayısal sütunlardaki (age, bmi, children, charges) aykırı değerleri ve dağılımı görselleştirmek için kutu grafikleri (boxplots) çizilir.

Dağılım Grafiği (Histogram ve KDE): charges sütununun dağılımını gösteren bir histogram ve yoğunluk tahmini (KDE - Kernel Density Estimate) çizilir. Başlıkta "Quality Distribution Plot" yazsa da, bu aslında 'charges' dağılımını göstermektedir.

Pair Plot: Veri çerçevesindeki tüm sayısal sütunlar arasındaki ikili ilişkileri (dağılımlar ve saçılım grafikleri) gösteren bir pair plot oluşturulur.

Saçılım Grafikleri:

age vs. charges

bmi vs. charges

children vs. charges
Bu grafikler, 'smoker' (sigara içen) durumuna göre renklendirilerek, sigara içmenin masraflar üzerindeki etkisini incelemeye yardımcı olur.

Korelasyon Isı Haritası: Sayısal sütunlar arasındaki korelasyonu gösteren bir ısı haritası çizilir. Bu, değişkenler arasındaki doğrusal ilişkilerin gücünü ve yönünü anlamak için faydalıdır.

In [11]:
# Kodlama ve Bölme
def encode_and_split(df, target='charges', test_size=0.2):
    df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True)
    X = df_encoded.drop(target, axis=1)
    y = df_encoded[target]
    return train_test_split(X, y, test_size=test_size, random_state=42)

X_train, X_test, y_train_raw, y_test_raw = encode_and_split(df_filtered)

print(X_train.head())


     age    bmi  children  sex_male  smoker_yes  region_northwest  \
203   27  36.08         0     False        True             False   
310   50  26.60         0      True       False             False   
491   61  25.08         0     False       False             False   
339   46  27.72         1     False       False             False   
10    25  26.22         0      True       False             False   

     region_southeast  region_southwest  
203              True             False  
310             False              True  
491              True             False  
339              True             False  
10              False             False  


Veri Bölme ve Dönüşüm
encode_and_split Fonksiyonu: Bu fonksiyon, verilen veri çerçevesini tekrar one-hot encoding'e tabi tutar, hedef değişkeni ayırır ve veriyi eğitim (X_train, y_train_raw) ve test (X_test, y_test_raw) setlerine böler.



In [12]:
# Log Dönüşümü
def log_transform(y):
    return np.log1p(y), np.expm1

y_train, inverse_transform = log_transform(y_train_raw)
y_test, _ = log_transform(y_test_raw)




 Log Dönüşümü: log_transform fonksiyonu, hedef değişken olan y'ye np.log1p (log(1+x)) dönüşümü uygular. Bu, genellikle sağa çarpık dağılıma sahip verilerde (örneğin, masraflar gibi) model performansını artırmak ve varsayımları karşılamak için yapılır. np.expm1 ise ters dönüşüm için bir referans olarak döndürülür.

 Yani, bu kullandığımız veri setindeki charges gibi finansal veya sayım verilerinde yaygın olarak kullanılan güçlü bir ön işleme adımıdır. Verinin dağılımını iyileştirerek, aykırı değerlerin etkisini azaltarak ve model varsayımlarını karşılayarak makine öğrenimi modellerinin performansını ve genellenebilirliğini artırır.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA Görselleştirmesi
pca = PCA(n_components=2)
X_pca = pca.fit_transform(scaler.fit_transform(X))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c='blue', alpha=0.5)
plt.title('PCA Görselleştirmesi')
plt.show()

In [14]:
#Modellerin Tanımı ve Eğitimi

models = {
    'Doğrusal Regresyon': Pipeline([
        ('ölçekleyici', StandardScaler()),
        ('model', LinearRegression())
    ]),
    'Ridge Regresyonu': Pipeline([
        ('ölçekleyici', StandardScaler()),
        ('model', Ridge(alpha=1.0))
    ]),
    'Rastgele Orman': Pipeline([
        ('model', RandomForestRegressor(n_estimators=100, random_state=42))
    ])
}

results = []

for name, pipeline in models.items():
    pipeline.fit(X_train, y_train)
    y_pred_log = pipeline.predict(X_test)
    y_pred = inverse_transform(y_pred_log)
    y_true = inverse_transform(y_test)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    results.append({'Model': name, 'RMSE': rmse, 'R2': r2})


In [15]:
# Örnek Model Sonuçları (örnek amaçlı, gerçek modeller burada eğitilmeli)
results = [
    {'Model': 'Linear Regression', 'RMSE': 4500, 'R2': 0.82},
    {'Model': 'Random Forest', 'RMSE': 3200, 'R2': 0.89},
    {'Model': 'Gradient Boosting', 'RMSE': 3100, 'R2': 0.90}
]


In [16]:

# Sonuç Karşılaştırması
results_df = pd.DataFrame(results)
print(results_df.sort_values(by='RMSE'))


               Model  RMSE    R2
2  Gradient Boosting  3100  0.90
1      Random Forest  3200  0.89
0  Linear Regression  4500  0.82


Örnek Model Sonuçları ve Karşılaştırma
Placeholder Sonuçlar: Bu bölüm, farklı makine öğrenimi modelleri (Lineer Regresyon, Random Forest, Gradient Boosting) için örnek RMSE (Ortalama Kare Hata Kökü) ve R2 (Belirleme Katsayısı) değerlerini içeren bir liste tanımlar. Bu değerler gerçek model eğitiminden gelmemektedir, sadece örnek amaçlıdır.

Sonuç Karşılaştırması: Tanımlanan örnek sonuçlar bir DataFrame'e dönüştürülür ve RMSE'ye göre sıralanır.


In [ ]:

# Grafiklerle görselleştirme
plt.figure(figsize=(8, 5))
sns.barplot(data=results_df, x='Model', y='RMSE', palette='coolwarm')
plt.title("Modellere Göre RMSE Karşılaştırması")
plt.ylabel("RMSE (Düşük = Daha İyi)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
sns.barplot(data=results_df, x='Model', y='R2', palette='viridis')
plt.title("Modellere Göre R2 Karşılaştırması")
plt.ylabel("R2 Skoru (Yüksek = Daha İyi)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Görselleştirme: RMSE ve R2 skorlarını modellere göre karşılaştıran çubuk grafikler (bar plots) çizilir. Bu grafikler, hangi modelin (örnek verilere göre) daha iyi performans gösterdiğini görsel olarak sunar.

In [18]:
from google.colab import output
output.clear()  # bu sadece anlık çıktıyı temizler

In [19]:
from google.colab import files
files.download("insurance.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>